# Text Summarization using Pegasus/xsum

$by: Jeremiah\space Chinyelugo$

In this notebook, we will be fine tuning a pretrained bart-large-cnn transformer model on the samsum dataset to perform a wide range of text summarization tasks.

#### Installing the kaggle notebook dependencies

In [1]:
# library & utility to support 7zip archive compression, decompression, encryption and decryption

! pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00


In [2]:
# the transformers library

! pip install transformers

In [3]:
# installing transformers directly from github to prevent any errors that might show up later

! pip install -U git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fssi3o2z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fssi3o2z
  Resolved https://github.com/huggingface/transformers.git to commit 8f093fb799246f7dd9104ff44728da0c53a9f67a
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7170473 sha256=5b0077d4a1d7a2fe9acb755edf7bfa35af364f648598cd199b35f48ee9054893
  Stored in directory: /tmp/pip-ephem-wheel-cache-e6zwc4xd/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.2

In [4]:
# hugginface accelerate

! pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-kz000zv3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-kz000zv3
  Resolved https://github.com/huggingface/accelerate.git to commit 665d5180fcc01d5700f7a9aa3f9bdb75c6055dce
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for accelerate: filename=accelerate-0.21.0.dev0-py3-none-any.whl size=228089 sha256=7ddc7f3655d9ac992b354b25eddf500677c7a14d5b3b1865c63e1014815d443f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_4gp50l8/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [5]:
pip install --upgrade accelerate

Note: you may need to restart the kernel to use updated packages.


In [6]:
# evaluate library

!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.8 MB/s eta 0:00:00


In [7]:
# rouge score metric

!pip install rouge_score

  Preparing metadata (setup.py) ... - \ done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=0b22ee0323023a35ae166588ef185f2369296f39647beeedac5471894c1fb2d8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [8]:
# tensorflow io

!pip install tensorflow_io==0.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow_io
    Found existing installation: tensorflow-io 0.31.0
    Uninstalling tensorflow-io-0.31.0:
      Successfully uninstalled tensorflow-io-0.31.0


#### Importing relevant packages

In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, pipeline
import nltk
import pandas as pd
import tensorflow as tf
import evaluate

In [10]:
# ensuring Kaggle GPU is being used 

gpus = tf.config.list_physical_devices('GPU')
gpus

tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print("GPU activated successfully.")

GPU activated successfully.


In [11]:
# monitoring the GPU

!nvidia-smi

Mon Jun 12 02:13:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Loading the dataset

In [12]:
# loading the samsum

dataset = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
# loading the model tokenizer

model_checkpoint = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
# loading the model

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [15]:
# viewing the dataset

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [16]:
# viewing 1 example from our train split

def show_examples(dataset, num_samples=1, seed=42):
    dataset = dataset['train'].shuffle(seed=seed).select(range(num_samples))
    for example in dataset:
        print(f">> Dialogue <<\n{example['dialogue']}\n")
        print(f">> Summary: {example['summary']}\n")

show_examples(dataset)

>> Dialogue <<
Lucy: omg did you see JK this morning?
Sue: I try to avoid it lol
Lucy: you should have seen it it was disgusting
Sue: I cant do it anymore i try to listen to the radio in the mornings.. jk makes you think the whole world is full of idiots lol
Lucy: you may be right I dont know how some of them can go on there in public for the world to see
Sue: I would die if I got a call to go on there lol
Sue: could you imagine ha ha 
Lucy: I would piss myself If I saw you and Andy up there
Sue: over my dead body !

>> Summary: Sue doesn't watch JK any more as it's disgusting.



#### Testing the model

When working on machine learning projects, it is always helpful to create or have a baseline metric to which you can compare you trained model performance to. In text summarization, a common baseline is to take the first 3 sentences to get a sense of the model's rouge metric. 

The dataset we are working on contains message like conversations and not articles which are much larger, so we will evaluate our pre-trained model on an instance rather than the first 3 sentences.

In [17]:
# our baseline instance

sequence = dataset['train'][10]['dialogue']
print(sequence)

Lucas: Hey! How was your day?
Demi: Hey there! 
Demi: It was pretty fine, actually, thank you!
Demi: I just got promoted! :D
Lucas: Whoa! Great news!
Lucas: Congratulations!
Lucas: Such a success has to be celebrated.
Demi: I agree! :D
Demi: Tonight at Death & Co.?
Lucas: Sure!
Lucas: See you there at 10pm?
Demi: Yeah! See you there! :D


In [18]:
# initializing the transformers pipeline. The pipeline taskes care of model checkpoints required to summarize our data

pipe = pipeline('summarization', model=model_checkpoint)

In [19]:
# pipeline output from baseline instance 

output = pipe(sequence)
output

Your max_length is set to 142, but your input_length is only 131. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)


[{'summary_text': '"I just got promoted! :D" "Such a success has to be celebrated" "Tonight at Death & Co?" "See you there at 10pm?" "Yeah! See you there!" "I\'ll be there" "I\'m going to be there! I\'m so excited!"'}]

Before computing our rouge metric on the model's generate summary, let's see what text our pre-trained model has generated

In [20]:
# viewing model's generated summary

print('Model Summary: ')
print(output[0]['summary_text'].replace('<n>', '\n'))

print('\nActual Summary: ')
print(dataset['train'][10]['summary'])

Model Summary: 
"I just got promoted! :D" "Such a success has to be celebrated" "Tonight at Death & Co?" "See you there at 10pm?" "Yeah! See you there!" "I'll be there" "I'm going to be there! I'm so excited!"

Actual Summary: 
Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.


The generated summary isn't good enough, it is also in first person. We can do better.

In [21]:
# loading the rouge metric

rouge_score = evaluate.load('rouge')

In [22]:
generated_summary = output[0]['summary_text']
reference_summary = dataset['train'][10]['summary']

scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores = {k:float(f"{v*100:.2f}") for k, v in scores.items()}
scores

{'rouge1': 22.22, 'rouge2': 11.54, 'rougeL': 22.22, 'rougeLsum': 22.22}

The metric's are decent, considering that the pre-trained model wasn't trained on this dataset. 

Before we proceed, we need to ensure that the model we will train by fine tuning is not biased towards giving summaries with less than words since this is a Text Summarization task. To address this, we will filter out summaries (if any) with less than 4 words.

In [23]:
# filter out summaries with less than four (4) words

dataset = dataset.filter(lambda x: len(x['summary'].split()) > 3)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The next step is to tokenize our input data and its labels. When tokenizing, it is advisable to set a max length for the input data so we don't exceed the maximum context size of the model. Also because our labels are text as well, we will need to tokenize them as well.

In [24]:
# creating the tokenization function

def tokenize_dataset(data):
    inputs = tokenizer(
        data['dialogue'], max_length=512,
        padding="max_length", truncation=True
    )
    
    labels = tokenizer(
        data['summary'], max_length=128,
        truncation=True,
    )
    
    inputs['labels'] = labels['input_ids']
    return inputs

In [25]:
# applying the function
 
tokenized_dataset = dataset.map(tokenize_dataset, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Fine tuning with keras

Since we have loaded our model tokenizer and checkpoint earlier in this notebook, we need to prepare our data for fine-tuning.

First, we will use the `DataCollatorForSeq2Seq` function to dynamically pad our inputs and labels to ensure all the instances in our datset have the same length.

In [26]:
from transformers import DataCollatorForSeq2Seq

# dynamically pads inputs and labels
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')

In [27]:
# removing column names from our tokenized dataset

tokenized_dataset = tokenized_dataset.remove_columns(
    dataset['train'].column_names
)

In [28]:
# converting the dataset to tensorflow objects

tf_train_dataset = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=4,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_dataset["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=4,
)

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [29]:
from transformers import create_optimizer
import tensorflow as tf

# defining training hyperparameters and compile

num_train_epochs = 2
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-6, num_warmup_steps=0,
    num_train_steps=num_train_steps, weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [30]:
# fitting the model

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, epochs=num_train_epochs
)

Epoch 1/2
3672/3672 [==============================] - 2643s 703ms/step - loss: 1.3719 - val_loss: 1.3795
Epoch 2/2
3672/3672 [==============================] - 2572s 701ms/step - loss: 1.1644 - val_loss: 1.3637


In [31]:
# saving our fine-tuned model

model.save_pretrained("first_model")

In [32]:
# saving our model tokenizer

tokenizer.save_pretrained("first_tokenizer")

('first_tokenizer/tokenizer_config.json',
 'first_tokenizer/special_tokens_map.json',
 'first_tokenizer/vocab.json',
 'first_tokenizer/merges.txt',
 'first_tokenizer/added_tokens.json',
 'first_tokenizer/tokenizer.json')